# 카카오 로컬 API -> 위도, 경도


In [5]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
filename

Saving kakaoapikey.txt to kakaoapikey (1).txt


'kakaoapikey.txt'

In [6]:
with open(filename) as f:
    api_key = f.read()    

In [7]:
len(api_key)

32

In [8]:
api_key

'eff2e11dd539b70fd84f03d14667d5c8'

In [9]:
import requests
from urllib.parse import quote

In [ ]:
addr = '서울특별시 중구 세종대로 110(태평로1가)'
search_url = "https://dapi.kakao.com/v2/local/search/address.json"
option = f'confmKey={api_key}&currentPage=1&countPerPage=10&keyword={quote(bldg)}'
url = f'{road_url}?{option}&resultType=json'

In [12]:
addr = '서울특별시 중구 세종대로 110(태평로1가)'
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{search_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A4%91%EA%B5%AC%20%EC%84%B8%EC%A2%85%EB%8C%80%EB%A1%9C%20110%28%ED%83%9C%ED%8F%89%EB%A1%9C1%EA%B0%80%29'

In [14]:
# Autho
result = requests.get(url,
                      headers="Authorization": f'kakaoAK {api_key}'}).json())
result

SyntaxError: ignored

In [17]:
result = requests.get(url,
            headers={"Authorization": f'KakaoAK {api_key}'}).json()
result

{'documents': [{'address': {'address_name': '서울 중구 태평로1가 31',
    'b_code': '1114010300',
    'h_code': '1114055000',
    'main_address_no': '31',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '명동',
    'region_3depth_name': '태평로1가',
    'sub_address_no': '',
    'x': '126.977829174031',
    'y': '37.5663174209601'},
   'address_name': '서울 중구 세종대로 110',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 중구 세종대로 110',
    'building_name': '서울특별시청',
    'main_building_no': '110',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_name': '태평로1가',
    'road_name': '세종대로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.977829174031',
    'y': '37.5663174209601',
    'zone_no': '04524'},
   'x': '126.977829174031',
   'y': '37.5663174209601'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [19]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
lat, lng

(37.5663174209601, 126.977829174031)

## 공공기관.csv파일에 위도, 경도 추가하기

In [20]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 공공기관.csv to 공공기관.csv


In [22]:
import pandas as pd
df = pd.read_csv(filename)
df

,공공기관,도로명주소
0,종로구청,서울특별시 종로구 종로1길 36(수송동)
1,노원구청,서울특별시 노원구 노해로 437(상계동)
2,송파구청,서울특별시 송파구 올림픽로 326(신천동)
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동)
4,강서구청,서울특별시 강서구 남부순환로 208(외발산동)


In [30]:
lat_list = []
lng_list = []

for i in range(5):
    addr = df.도로명주소[i]
    url = f'{search_url}?query={quote(addr)}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng )

In [31]:
df['위도'] = lat_list
df['경도'] = lng_list

In [32]:
df

,공공기관,도로명주소,위도,경도
0,종로구청,서울특별시 종로구 종로1길 36(수송동),37.573505,126.978988
1,노원구청,서울특별시 노원구 노해로 437(상계동),37.654519,127.056297
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),37.514477,127.105860
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),37.563426,126.903357
4,강서구청,서울특별시 강서구 남부순환로 208(외발산동),37.546867,126.820414


In [34]:
df.to_csv('공공기관2.csv', index=False)

## 지도위에 공공기관 표시하기

In [35]:
import folium

In [38]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시 주요 공원현황.csv to 서울시 주요 공원현황.csv


In [40]:
parks = pd.read_csv(filename, encoding='EUC-KR')
parks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 non-null    